# About this Notebook

Data Wrangling can be defined as the process to:
* gather
* assess
* clean
data in support of data analytics tasks.

This notebook contains the code to extract data related to Master Swim Performances and save it in a local format that supports Data Exploration activities.

## Supported Data Sources

Currently data is only extracted from the Canadian swimming site that collects the results for Masters Swimmers.

## Error Handling

When I created the meet list for a complete season, I found that some meets don't have results posted

Right now, my code assumes that results will exist.  This will need to be updated

In [3]:
import requests
import lxml.html as lh
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import time
from random import randint
import logging
import utilities as util

In [2]:
#set up the logger and initially set the logging level to debug
# since I am using a notebook, I am not going to capture the log in a file
logging.basicConfig(level=logging.DEBUG)

# Canadian Master Swimming Results Collection 

Starting with the information I know how to access from personal experience.

## Gather - www.swimming.ca
The following code will scrape the available data from the www.swimming.ca website for Masters Swimmers. The code is based on the website structure that exists during the time period Aug - Oct 2020.

The website has a layered approach to finding results.
1. Navigate to the page that has a list of meet-results
2. Filter the list of meet results by **Season**, **Province**, **Month**
3. Select the meet of interest in the filtered list
4. At this point it is possible to see all the results for each club that participated, or the results by event.  I will be collecting all the results by participating club.

I did contact Swim Canada to see if there was an API that would allow me to use an approach other than web scraping to get the results, but so far they have not been able to support my request.

For the parameters **Seasion**, **Province** and **Month**, there are few points to note:
* Season - the swim season in Canada is split over 2 calendar years and runs Sep to Aug.  So the swim season 2019/2020 will be the period 1 Sep 2019 to 31 Aug 2020.  The value passed to the **Season** parameter is the last 2 numbers in the second part of the season identifier.  So to access swim meets from Nov 2019, the season parameter will be 20 ('season'='20')
* Month - the month must be specified as a number from 1 to 12.  It is not possible to have all the months in a single season being available at one time
* Province - It is possible to filter by province/territory or use all provinces. To access an individual province/territory, the provinces/territories are numbered from 1 to 11 in alphabetical order.  So to get only Alberta, 'province'='1' and to get Yukon, 'province'='11'.  The supported Territories are Northwest Territories and Yukon.  For all provinces, no number is provided to province (province = '')

Since I am only interested in Masters results, when looking at the meet list, I will look for MEET TYPE  of Masters.  This will miss some swimmers that compete in non-Masters meets as an Open Master.  I plan to come back and find these results after collecting the Masters Meets

The only URL specified in my code is the URL associated with step 1 above, navigating to the meet-results page.  The subsequent URLs will be recovered from the information available when the various filters are applied.

In [3]:
# URLs of interest
url_meetlist ='https://www.swimming.ca/en/events-results/meet-results/'

In [ ]:
#change the logging level
logging.getLogger().setLevel(logging.INFO)

## Get Meet Info
The next section of code is used to retrieve meet information and write the results to a csv.  One csv per season

### Seasons Processed
To see what seasons have been processed, look at the csv's in the same file folder as this notebook.  Best effort list as I process a season:
* 2019-2020
* 2018-2019
* 2017-2018
* 2016-2017

For each season:

* update the season value and csv name in the dictionary
* change the returned list to a datafram
* write the dataframe to a csv

In [4]:
season_dict = {}
season_dict['season'] = '19'
season_dict['meet_file'] = 'meets_2018_2019.csv'
season_dict['club_file'] = 'clubs_2018_2019.csv'
season_dict['swimmers_file'] = 'swimmers_2018_2019.csv'
season_dict['race_file'] = 'races_2018_2019.csv'

In [ ]:
MeetListFn = []

#get the meet list for a season
monthList = ['9', '10', '11', '12', '1', '2', '3', '4', '5', '6', '7', '8']
for mon in monthList:
    logging.info("month being processed %s", mon)
    tempList = util.getMeetList(season_dict['season'], mon, url_meetlist)
    for item in tempList:
        MeetListFn.append(item)

In [ ]:
len(MeetListFn)

In [ ]:
#create a dataframe and write the dataframe to a csv
df = pd.DataFrame(MeetListFn)
df.head()

In [ ]:
df.to_csv(season_dict['meet_file'], index=False)

## Get teams participating in meet info

Next section will get the team lists for each meet and write the results to a csv

In [5]:
# read in the meet list file
#df_meet = pd.read_csv('meets_2018_2019.csv')
df_meet = pd.read_csv(season_dict['meet_file'])
df_meet.head()

,meet_course,meet_date,meet_host,meet_prov,meet_status,meet_type,meet_url
0,SC,27 Oct 2018,BRO,QC,Completed,Masters,https://www.swimming.ca/en/meet/32167
1,SC,27 Oct 2018,HALT,NS,Completed,Masters,https://www.swimming.ca/en/meet/32128
2,SC,13 Oct 2018,CMSC,AB,Completed,Masters,https://www.swimming.ca/en/meet/24715
3,SC,25 Nov 2018,EMSC,AB,Completed,Masters,https://www.swimming.ca/en/meet/32275
4,SC,24 Nov 2018,SDSC,PE,Sanctioned,Masters,https://www.swimming.ca/en/meet/32333


In [6]:
df_meet.shape

(36, 7)

In [7]:
#make a list of the meet urls
meetURLS = df_meet['meet_url'].tolist()

In [ ]:
#change the logging level
#logging.getLogger().setLevel(logging.DEBUG)

In [8]:
clubListFn = []

for meet in meetURLS:
    logging.info("meet URL %s", meet)
    tempList = util.getTeamList(meet)
    for item in tempList:
        clubListFn.append(item)


INFO:root:meet URL https://www.swimming.ca/en/meet/32167
DEBUG:root:https://www.swimming.ca/en/meet/32167
DEBUG:root:sleep time is 14 seconds
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.swimming.ca:443
DEBUG:urllib3.connectionpool:https://www.swimming.ca:443 "GET /en/meet/32167 HTTP/1.1" 301 None
DEBUG:urllib3.connectionpool:https://www.swimming.ca:443 "GET /en/meet/32167/ HTTP/1.1" 200 None
DEBUG:root:https://www.swimming.ca/en/meet/32167/
DEBUG:root:Did I get here?
INFO:root:meet URL https://www.swimming.ca/en/meet/32128
DEBUG:root:https://www.swimming.ca/en/meet/32128
DEBUG:root:sleep time is 13 seconds
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.swimming.ca:443
DEBUG:urllib3.connectionpool:https://www.swimming.ca:443 "GET /en/meet/32128 HTTP/1.1" 301 None
DEBUG:urllib3.connectionpool:https://www.swimming.ca:443 "GET /en/meet/32128/ HTTP/1.1" 200 None
DEBUG:root:https://www.swimming.ca/en/meet/32128/
DEBUG:root:Did I get here?
INFO:roo

DEBUG:urllib3.connectionpool:https://www.swimming.ca:443 "GET /en/meet/32460/ HTTP/1.1" 200 None
DEBUG:root:https://www.swimming.ca/en/meet/32460/
DEBUG:root:Did I get here?
INFO:root:meet URL https://www.swimming.ca/en/meet/32577
DEBUG:root:https://www.swimming.ca/en/meet/32577
DEBUG:root:sleep time is 14 seconds
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.swimming.ca:443
DEBUG:urllib3.connectionpool:https://www.swimming.ca:443 "GET /en/meet/32577 HTTP/1.1" 301 None
DEBUG:urllib3.connectionpool:https://www.swimming.ca:443 "GET /en/meet/32577/ HTTP/1.1" 200 None
DEBUG:root:https://www.swimming.ca/en/meet/32577/
DEBUG:root:Did I get here?
INFO:root:meet URL https://www.swimming.ca/en/meet/32561
DEBUG:root:https://www.swimming.ca/en/meet/32561
DEBUG:root:sleep time is 10 seconds
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.swimming.ca:443
DEBUG:urllib3.connectionpool:https://www.swimming.ca:443 "GET /en/meet/32561 HTTP/1.1" 301 None
DEBUG:ur

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.swimming.ca:443
DEBUG:urllib3.connectionpool:https://www.swimming.ca:443 "GET /en/meet/34147 HTTP/1.1" 301 None
DEBUG:urllib3.connectionpool:https://www.swimming.ca:443 "GET /en/meet/34147/ HTTP/1.1" 200 None
DEBUG:root:https://www.swimming.ca/en/meet/34147/
DEBUG:root:Did I get here?
INFO:root:meet URL https://www.swimming.ca/en/meet/34001
DEBUG:root:https://www.swimming.ca/en/meet/34001
DEBUG:root:sleep time is 11 seconds
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.swimming.ca:443
DEBUG:urllib3.connectionpool:https://www.swimming.ca:443 "GET /en/meet/34001 HTTP/1.1" 301 None
DEBUG:urllib3.connectionpool:https://www.swimming.ca:443 "GET /en/meet/34001/ HTTP/1.1" 200 None
DEBUG:root:https://www.swimming.ca/en/meet/34001/
DEBUG:root:Did I get here?
INFO:root:meet URL https://www.swimming.ca/en/meet/34007
DEBUG:root:https://www.swimming.ca/en/meet/34007
DEBUG:root:sleep time is 11 seconds
DEBUG:ur

In [9]:
#create a dataframe and write the dataframe to a csv
club_df = pd.DataFrame(clubListFn)
club_df.head()

,club_name,club_number,club_res_url
0,La Vague de Brossard,73992,https://www.swimming.ca/en/meet/611459/?factio...
1,C.N. Mustang Boucherville,72337,https://www.swimming.ca/en/meet/611459/?factio...
2,Club Aquatique Montreal,72346,https://www.swimming.ca/en/meet/611459/?factio...
3,Maitres A Contre-Courant,74019,https://www.swimming.ca/en/meet/611459/?factio...
4,McGill Masters,76063,https://www.swimming.ca/en/meet/611459/?factio...


In [10]:
#club_df.to_csv('clubs_2018_2019.csv', index=False)
club_df.to_csv(season_dict['club_file'], index=False)

## Get swimmer and race info

The next section will get the results for meet and save the information to two files.  One will contain the information on the swimmer and one will contain the informaiton on the races

In [11]:
season_dict

{'season': '19',
 'meet_file': 'meets_2018_2019.csv',
 'club_file': 'clubs_2018_2019.csv',
 'swimmers_file': 'swimmers_2018_2019.csv',
 'race_file': 'races_2018_2019.csv'}

In [12]:
#get the csv containing the club links
swims_df = pd.read_csv(season_dict['club_file'])

In [13]:
swims_df.shape

(555, 3)

In [14]:
swims_df.head()

,club_name,club_number,club_res_url
0,La Vague de Brossard,73992.0,https://www.swimming.ca/en/meet/611459/?factio...
1,C.N. Mustang Boucherville,72337.0,https://www.swimming.ca/en/meet/611459/?factio...
2,Club Aquatique Montreal,72346.0,https://www.swimming.ca/en/meet/611459/?factio...
3,Maitres A Contre-Courant,74019.0,https://www.swimming.ca/en/meet/611459/?factio...
4,McGill Masters,76063.0,https://www.swimming.ca/en/meet/611459/?factio...


In [15]:
len(swims_df['club_name'].unique())

153

In [16]:
#make a list of urls for the results from each club
clubURLS = swims_df['club_res_url'].tolist()

In [17]:
len(clubURLS)

555

In [18]:
sw_list = []
rc_list = []

In [19]:
for item in clubURLS:
    try:
        if "https" in item:
            tmp_sw_list, tmp_rc_list = util.getRaceResults(item)
            sw_list += tmp_sw_list
            rc_list += tmp_rc_list
    except:
        print(item, "is not a valid url")

DEBUG:root:https://www.swimming.ca/en/meet/611459/?faction=73992
DEBUG:root:sleep time is 13 seconds
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.swimming.ca:443
DEBUG:urllib3.connectionpool:https://www.swimming.ca:443 "GET /en/meet/611459/?faction=73992 HTTP/1.1" 200 None
DEBUG:root:https://www.swimming.ca/en/meet/611459/?faction=73992
DEBUG:root:number of tables 5
DEBUG:root:on table 0
DEBUG:root:first row of table, ignore
DEBUG:root:on table 1
DEBUG:root:first row of table, ignore
DEBUG:root:on table 2
DEBUG:root:first row of table, ignore
DEBUG:root:https://www.swimming.ca/en/meet/611459/?faction=72337
DEBUG:root:sleep time is 12 seconds
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.swimming.ca:443
DEBUG:urllib3.connectionpool:https://www.swimming.ca:443 "GET /en/meet/611459/?faction=72337 HTTP/1.1" 200 None
DEBUG:root:https://www.swimming.ca/en/meet/611459/?faction=72337
DEBUG:root:number of tables 5
DEBUG:root:on table 0
DEBUG:root:fir

DEBUG:root:https://www.swimming.ca/en/meet/611219/?faction=90172
DEBUG:root:number of tables 3
DEBUG:root:on table 0
DEBUG:root:first row of table, ignore
DEBUG:root:https://www.swimming.ca/en/meet/611219/?faction=81783
DEBUG:root:sleep time is 10 seconds
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.swimming.ca:443
DEBUG:urllib3.connectionpool:https://www.swimming.ca:443 "GET /en/meet/611219/?faction=81783 HTTP/1.1" 200 None
DEBUG:root:https://www.swimming.ca/en/meet/611219/?faction=81783
DEBUG:root:number of tables 3
DEBUG:root:on table 0
DEBUG:root:first row of table, ignore
DEBUG:root:https://www.swimming.ca/en/meet/611219/?faction=90648
DEBUG:root:sleep time is 11 seconds
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.swimming.ca:443
DEBUG:urllib3.connectionpool:https://www.swimming.ca:443 "GET /en/meet/611219/?faction=90648 HTTP/1.1" 200 None
DEBUG:root:https://www.swimming.ca/en/meet/611219/?faction=90648
DEBUG:root:number of tables 3
D

DEBUG:root:sleep time is 12 seconds
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.swimming.ca:443
DEBUG:urllib3.connectionpool:https://www.swimming.ca:443 "GET /en/meet/611217/?faction=81791 HTTP/1.1" 200 None
DEBUG:root:https://www.swimming.ca/en/meet/611217/?faction=81791
DEBUG:root:number of tables 3
DEBUG:root:on table 0
DEBUG:root:first row of table, ignore
DEBUG:root:https://www.swimming.ca/en/meet/612025/?faction=73928
DEBUG:root:sleep time is 10 seconds
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.swimming.ca:443
DEBUG:urllib3.connectionpool:https://www.swimming.ca:443 "GET /en/meet/612025/?faction=73928 HTTP/1.1" 200 None
DEBUG:root:https://www.swimming.ca/en/meet/612025/?faction=73928
DEBUG:root:number of tables 5
DEBUG:root:on table 0
DEBUG:root:first row of table, ignore
DEBUG:root:on table 1
DEBUG:root:first row of table, ignore
DEBUG:root:on table 2
DEBUG:root:first row of table, ignore
DEBUG:root:https://www.swimming.ca/en/mee

DEBUG:root:sleep time is 11 seconds
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.swimming.ca:443
DEBUG:urllib3.connectionpool:https://www.swimming.ca:443 "GET /en/meet/611825/?faction=76060 HTTP/1.1" 200 None
DEBUG:root:https://www.swimming.ca/en/meet/611825/?faction=76060
DEBUG:root:number of tables 5
DEBUG:root:on table 0
DEBUG:root:first row of table, ignore
DEBUG:root:on table 1
DEBUG:root:first row of table, ignore
DEBUG:root:on table 2
DEBUG:root:first row of table, ignore
DEBUG:root:https://www.swimming.ca/en/meet/611825/?faction=89263
DEBUG:root:sleep time is 12 seconds
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.swimming.ca:443
DEBUG:urllib3.connectionpool:https://www.swimming.ca:443 "GET /en/meet/611825/?faction=89263 HTTP/1.1" 200 None
DEBUG:root:https://www.swimming.ca/en/meet/611825/?faction=89263
DEBUG:root:number of tables 5
DEBUG:root:on table 0
DEBUG:root:first row of table, ignore
DEBUG:root:on table 1
DEBUG:root:first ro

DEBUG:root:first row of table, ignore
DEBUG:root:https://www.swimming.ca/en/meet/611825/?faction=73973
DEBUG:root:sleep time is 13 seconds
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.swimming.ca:443
DEBUG:urllib3.connectionpool:https://www.swimming.ca:443 "GET /en/meet/611825/?faction=73973 HTTP/1.1" 200 None
DEBUG:root:https://www.swimming.ca/en/meet/611825/?faction=73973
DEBUG:root:number of tables 3
DEBUG:root:on table 0
DEBUG:root:first row of table, ignore
DEBUG:root:https://www.swimming.ca/en/meet/611825/?faction=87439
DEBUG:root:sleep time is 13 seconds
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.swimming.ca:443
DEBUG:urllib3.connectionpool:https://www.swimming.ca:443 "GET /en/meet/611825/?faction=87439 HTTP/1.1" 200 None
DEBUG:root:https://www.swimming.ca/en/meet/611825/?faction=87439
DEBUG:root:number of tables 3
DEBUG:root:on table 0
DEBUG:root:first row of table, ignore
DEBUG:root:https://www.swimming.ca/en/meet/611825/?faction

...

DEBUG:root:first row of table, ignore
DEBUG:root:https://www.swimming.ca/en/meet/615793/?faction=78522
DEBUG:root:sleep time is 14 seconds
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.swimming.ca:443
DEBUG:urllib3.connectionpool:https://www.swimming.ca:443 "GET /en/meet/615793/?faction=78522 HTTP/1.1" 200 None
DEBUG:root:https://www.swimming.ca/en/meet/615793/?faction=78522
DEBUG:root:number of tables 3
DEBUG:root:on table 0
DEBUG:root:first row of table, ignore
DEBUG:root:https://www.swimming.ca/en/meet/615793/?faction=100
DEBUG:root:sleep time is 13 seconds
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.swimming.ca:443
DEBUG:urllib3.connectionpool:https://www.swimming.ca:443 "GET /en/meet/615793/?faction=100 HTTP/1.1" 200 None
DEBUG:root:https://www.swimming.ca/en/meet/615793/?faction=100
DEBUG:root:number of tables 3
DEBUG:root:on table 0
DEBUG:root:first row of table, ignore
DEBUG:root:https://www.swimming.ca/en/meet/616304/?faction=73869

nan is not a valid url


DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.swimming.ca:443
DEBUG:urllib3.connectionpool:https://www.swimming.ca:443 "GET /en/meet/616304/?faction=73869 HTTP/1.1" 200 None
DEBUG:root:https://www.swimming.ca/en/meet/616304/?faction=73869
DEBUG:root:number of tables 3
DEBUG:root:on table 0
DEBUG:root:first row of table, ignore
DEBUG:root:https://www.swimming.ca/en/meet/616238/?faction=82103
DEBUG:root:sleep time is 10 seconds
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.swimming.ca:443
DEBUG:urllib3.connectionpool:https://www.swimming.ca:443 "GET /en/meet/616238/?faction=82103 HTTP/1.1" 200 None
DEBUG:root:https://www.swimming.ca/en/meet/616238/?faction=82103
DEBUG:root:number of tables 5
DEBUG:root:on table 0
DEBUG:root:first row of table, ignore
DEBUG:root:on table 1
DEBUG:root:first row of table, ignore
DEBUG:root:on table 2
DEBUG:root:first row of table, ignore
DEBUG:root:https://www.swimming.ca/en/meet/616238/?faction=76185
DEBUG:root:s

nan is not a valid url


DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.swimming.ca:443
DEBUG:urllib3.connectionpool:https://www.swimming.ca:443 "GET /en/meet/616037/?faction=73973 HTTP/1.1" 200 None
DEBUG:root:https://www.swimming.ca/en/meet/616037/?faction=73973
DEBUG:root:number of tables 3
DEBUG:root:on table 0
DEBUG:root:first row of table, ignore


In [20]:
swim_df = pd.DataFrame(sw_list)
swim_df.head()

,sw_gender,sw_id,sw_name,sw_url,sw_yob
0,male,5256169,"Beaudoin, Mathieu",https://www.swimming.ca/en/swimmer/5256169/,1977
1,male,4204240,"Berger, Christian",https://www.swimming.ca/en/swimmer/4204240/,1944
2,male,4537977,"Botbol, Jacques",https://www.swimming.ca/en/swimmer/4537977/,1951
3,male,5256229,"Bourdages, Sylvain",https://www.swimming.ca/en/swimmer/5256229/,1958
4,male,4204241,"Dussault, Pierre",https://www.swimming.ca/en/swimmer/4204241/,1945


In [21]:
swim_df.shape

(3786, 5)

In [22]:
race_df = pd.DataFrame(rc_list)
race_df.head()

,rc_course,rc_dist_stroke,rc_round,rc_time,sw_gender,sw_id,sw_yob
0,25m,50m Freestyle,Final,33.33,male,5256169,1977
1,25m,100m Freestyle,Final,1:13.84,male,5256169,1977
2,25m,50m Backstroke,Final,54.80,male,4204240,1944
3,25m,50m Breaststroke,Final,57.01,male,4204240,1944
4,25m,50m Butterfly,Final,57.64,male,4204240,1944


In [23]:
race_df.shape

(18723, 7)

In [24]:
swim_df.to_csv(season_dict['swimmers_file'], index=False)
race_df.to_csv(season_dict['race_file'], index=False)

## Assess - www.swimming.ca

## Clean - www.swimming.ca

# Development Code

While creating the final functions, I used the following code to explore the webpages to set up the parsing correctly

I don't want to loose it, but it should not be run

### Explore Structure of html pages
In order to recover the information of interest, the way the information is presented on the various html pages needs to be understood.

Once I know how to capture the data of interest, I will create functions to iterate through diffferent seasons, months and meets.

The information I am interested in can be grouped into 3 categories.

**Swimmer Information**
* sw_id_num - unique swimmer id number
* sw_name - swimmers name
* sw_yob - swimmers year of birth

Note that swimmers age is determined by the age on Dec 31st of the year in which the competition happened.

**Meet Information**

* mt_id_num - unique meet id number
* mt_name - meet name
* mt_date - first day of meet
* mt_duration - number of days the swim meet lasted
* mt_pool_name - name of the pool
* mt_pool_address - address of the pool
* mt_sc_lc - indication if meet was a long course or short course meet

**Race Information**
* rc_dist - distance of race - eg 25m, 100m, 400m
* rc_stroke - stroke of race - freestyle, backstroke, butterfly, breaststroke, IM
* rc_time - time it took the swimmer to complete the distance
* rc_round - heat, final, split of longer swim or part of relay
* sw_id_num - unique swimmer id
* mt_id_num - unique meet id

I am also confident that at some point I will want club information, but at this point I'm not sure what I would do with it.

I also have not decided the best way to store the information I scrape.  Initially, I will put it in 3 different csv's, but would like to explore a graph database.  I have not used them before and would like to learn.

In [ ]:
# start with the 2019/2020 season, noting that the season came to an abrupt end in March 2020 because of Covid19
# using November since I know there is a masters meet in Nov 2019
call_params = {'season':'20', 'province':'', 'month':'11'}
#response=requests.get('https://www.swimming.ca/en/events-results/meet-results/', params={'season':'20', 'province':'', 'month':'11'})
response=requests.get('https://www.swimming.ca/en/events-results/meet-results/', params=call_params)
#sanity check that my parameter specification worked as expected
response.url

In [ ]:
#use Beautiful Soup to parse the returned page
meetList_resp = BeautifulSoup(response.text, 'lxml')

In [ ]:
meetList_resp.find_all('tr')

In [ ]:
#find the masters swim meets in the response text
meetList = []
for item in meetList_resp.find_all('tr'):
    if item.contents[5].contents[0] == "Masters":
        temp_dict = {}
        temp_dict['meet_date'] = item.contents[0].contents[0].contents[0]
        temp_dict['meet_url'] = item.contents[1].a.attrs['href']
        temp_dict['meet_prov'] = item.contents[2].contents[0]
        temp_dict['meet_host'] = item.contents[3].contents[0].contents[0]
        temp_dict['meet_course'] = item.contents[4].contents[0]
        temp_dict['meet_type'] = item.contents[5].contents[0]
        meetList.append(temp_dict)
        ##used for debug/development
        #print(item.contents[0].contents[0].contents[0])
        #print(item.contents[1].a.attrs['href'])
        #print(item.contents[2].contents[0])
        #print(item.contents[3].contents[0].contents[0])
        #print(item.contents[4].contents[0])
        #print(item.contents[5].contents[0])
print("number of masters meets", len(meetList))

In [ ]:
meetList[0]

So 8 masters meets were held in Canada in Nov 2019.

Now use the meet_url to get the results for that meet.  The use Beautiful Soup to parse the returned page

In [4]:
## try to fix the gender and get the meet id's
meet_tmp_url = "https://www.swimming.ca/en/meet/20064"
club_gold_tmp_url = "https://www.swimming.ca/en/meet/597428/?faction=74044"
club_pa_tmp_url = "https://www.swimming.ca/en/meet/597428/?faction=81908"
club_wrose_tmp_url = "https://www.swimming.ca/en/meet/598224/?faction=88757"
club_vb_tmp_url = "https://www.swimming.ca/en/meet/617956/?faction=73992"

In [19]:
#get meetpage
r3 = requests.get(meet_tmp_url)
temp_mt_rsp = BeautifulSoup(r3.text, 'lxml')

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.swimming.ca:443
DEBUG:urllib3.connectionpool:https://www.swimming.ca:443 "GET /en/meet/20064 HTTP/1.1" 301 None
DEBUG:urllib3.connectionpool:https://www.swimming.ca:443 "GET /en/meet/20064/ HTTP/1.1" 200 None


In [20]:
#get the 2 meet id's
meetid_1 = meet_tmp_url.split("/")[-1]
meetid_1

'20064'

In [21]:
temp_mt_rsp.find_all('option')

[<option value="">Participants</option>,
 <option data-href="https://www.swimming.ca/en/meet/597428/?faction=74044" value="74044">Saskatoon Goldfins Swim Club</option>,
 <option data-href="https://www.swimming.ca/en/meet/597428/?faction=76612" value="76612">Regina Masters Swim Club</option>,
 <option data-href="https://www.swimming.ca/en/meet/597428/?faction=81908" value="81908">Prince Albert Masters</option>,
 <option value="">Men’s Events</option>,
 <option data-href="https://www.swimming.ca/en/meet/597428/?gender=M&amp;competition=Style-1" value="Style-1">50m Freestyle</option>,
 <option data-href="https://www.swimming.ca/en/meet/597428/?gender=M&amp;competition=Style-3" value="Style-3">200m Freestyle</option>,
 <option data-href="https://www.swimming.ca/en/meet/597428/?gender=M&amp;competition=Style-9" value="Style-9">50m Backstroke</option>,
 <option data-href="https://www.swimming.ca/en/meet/597428/?gender=M&amp;competition=Style-12" value="Style-12">50m Breaststroke</option>,
 <

In [26]:
for item in temp_mt_rsp.find_all('option'):
    #print(item.contents[0])
    temp_dict = {}
    if "Events" in item.contents[0]:
        #print(item)
        break
    elif "Participants" not in item.contents[0]:
        temp_dict['club_res_url'] = item.attrs['data-href']
        temp_dict['club_number'] = item.attrs['value']
        temp_dict['club_name'] = item.contents[0]
        temp_dict['meetid2'] = item.attrs['data-href'].split("/")[-2]
        #clubList.append(temp_dict)
        ## used for dev/debug
        print(item.attrs['data-href'])
        print(item.attrs['value'])
        print(item.contents[0])
        print(item.attrs['data-href'].split("/")[-2])

https://www.swimming.ca/en/meet/597428/?faction=74044
74044
Saskatoon Goldfins Swim Club
597428
https://www.swimming.ca/en/meet/597428/?faction=76612
76612
Regina Masters Swim Club
597428
https://www.swimming.ca/en/meet/597428/?faction=81908
81908
Prince Albert Masters
597428


In [27]:
#get club 1 page
r4 = requests.get(club_gold_tmp_url)
temp_gold_rsp = BeautifulSoup(r4.text, 'lxml')

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.swimming.ca:443
DEBUG:urllib3.connectionpool:https://www.swimming.ca:443 "GET /en/meet/597428/?faction=74044 HTTP/1.1" 200 None


In [28]:
r5 = requests.get(club_pa_tmp_url)
temp_pa_rsp = BeautifulSoup(r5.text, 'lxml')

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.swimming.ca:443
DEBUG:urllib3.connectionpool:https://www.swimming.ca:443 "GET /en/meet/597428/?faction=81908 HTTP/1.1" 200 None


In [60]:
r6 = requests.get(club_wrose_tmp_url)
temp_wrose_rsp = BeautifulSoup(r6.text, 'lxml')

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.swimming.ca:443
DEBUG:urllib3.connectionpool:https://www.swimming.ca:443 "GET /en/meet/598224/?faction=88757 HTTP/1.1" 200 None


In [5]:
r7 = requests.get(club_vb_tmp_url)
temp_wrose_rsp = BeautifulSoup(r7.text, 'lxml')

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.swimming.ca:443
DEBUG:urllib3.connectionpool:https://www.swimming.ca:443 "GET /en/meet/617956/?faction=73992 HTTP/1.1" 200 None


In [6]:
#now I need to try and recover the gender from the table queries
temp_table = temp_wrose_rsp.find_all("table")
len(temp_table)

5

So the code below will find the Men, Women and Relay's in the table header (the print line with the table header text prefix)
it does not distinguish between mens and women's relays
the 2nd last table will be Resources and the last tabel will error since the option attribute does not exist

In [28]:
print("table header text", temp_table[0].find('caption').contents[0].split()[0])

table header text Men


In [21]:
# this will get the gender for the relays, but I'm not sure how to deal with it well in my utilities file
# I am also not sure I care if I have the gender for the relays, will leave it out for now
print("number of sub tables",len(temp_table[2].find_all('tbody')))
print("maybe header row", temp_table[2].find_all('tbody')[1].find('th').contents[0])

number of sub tables 2
maybe header row Women


In [29]:
from importlib import reload

In [36]:
reload(util)
tmp_sw, tmp_rc = util.getRaceResults(club_vb_tmp_url)

DEBUG:root:https://www.swimming.ca/en/meet/617956/?faction=73992
DEBUG:root:sleep time is 12 seconds
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.swimming.ca:443
DEBUG:urllib3.connectionpool:https://www.swimming.ca:443 "GET /en/meet/617956/?faction=73992 HTTP/1.1" 200 None
DEBUG:root:https://www.swimming.ca/en/meet/617956/?faction=73992
DEBUG:root:number of tables 5
DEBUG:root:on table 0
DEBUG:root:recovered gender from table head Men
DEBUG:root:first row of table, ignore
DEBUG:root:on table 1
DEBUG:root:recovered gender from table head Women
DEBUG:root:first row of table, ignore
DEBUG:root:on table 2
DEBUG:root:recovered gender from table head Relays
DEBUG:root:first row of table, ignore


In [38]:
tmp_sw[-1]

{'sw_url': None,
 'sw_id': None,
 'sw_name': None,
 'sw_yob': None,
 'sw_gender': 'Relays',
 'meet_id_2': '617956'}

In [32]:
tmp_rc[0]

{'rc_dist_stroke': '50m Freestyle',
 'rc_round': 'Final',
 'rc_time': '32.37',
 'rc_course': '25m',
 'sw_id': '5068464',
 'sw_yob': '2000',
 'sw_gender': 'Men',
 'meet_id_2': '617956'}

In [39]:
tmp_rc

[{'rc_dist_stroke': '50m Freestyle',
  'rc_round': 'Final',
  'rc_time': '32.37',
  'rc_course': '25m',
  'sw_id': '5068464',
  'sw_yob': '2000',
  'sw_gender': 'Men',
  'meet_id_2': '617956'},
 {'rc_dist_stroke': '100m Freestyle',
  'rc_round': 'Final',
  'rc_time': '1:20.47',
  'rc_course': '25m',
  'sw_id': '5068464',
  'sw_yob': '2000',
  'sw_gender': 'Men',
  'meet_id_2': '617956'},
 {'rc_dist_stroke': '50m Freestyle',
  'rc_round': 'Final',
  'rc_time': '33.16',
  'rc_course': '25m',
  'sw_id': '5256169',
  'sw_yob': '1977',
  'sw_gender': 'Men',
  'meet_id_2': '617956'},
 {'rc_dist_stroke': '50m Breaststroke',
  'rc_round': 'Final',
  'rc_time': '40.84',
  'rc_course': '25m',
  'sw_id': '5256169',
  'sw_yob': '1977',
  'sw_gender': 'Men',
  'meet_id_2': '617956'},
 {'rc_dist_stroke': '50m Breaststroke',
  'rc_round': 'Final',
  'rc_time': '57.00',
  'rc_course': '25m',
  'sw_id': '4204240',
  'sw_yob': '1944',
  'sw_gender': 'Men',
  'meet_id_2': '617956'},
 {'rc_dist_stroke': '

In [35]:
tmp_sw

[{'sw_url': 'https://www.swimming.ca/en/swimmer/5068464/',
  'sw_id': '5068464',
  'sw_name': 'Andreev, Yoan',
  'sw_yob': '2000',
  'sw_gender': 'Men',
  'meet_id_2': '617956'},
 {'sw_url': 'https://www.swimming.ca/en/swimmer/5256169/',
  'sw_id': '5256169',
  'sw_name': 'Beaudoin, Mathieu',
  'sw_yob': '1977',
  'sw_gender': 'Men',
  'meet_id_2': '617956'},
 {'sw_url': 'https://www.swimming.ca/en/swimmer/4204240/',
  'sw_id': '4204240',
  'sw_name': 'Berger, Christian',
  'sw_yob': '1944',
  'sw_gender': 'Men',
  'meet_id_2': '617956'},
 {'sw_url': 'https://www.swimming.ca/en/swimmer/5032276/',
  'sw_id': '5032276',
  'sw_name': 'Buckley, Kevin',
  'sw_yob': '1982',
  'sw_gender': 'Men',
  'meet_id_2': '617956'},
 {'sw_url': 'https://www.swimming.ca/en/swimmer/4204241/',
  'sw_id': '4204241',
  'sw_name': 'Dussault, Pierre',
  'sw_yob': '1945',
  'sw_gender': 'Men',
  'meet_id_2': '617956'},
 {'sw_url': 'https://www.swimming.ca/en/swimmer/5396522/',
  'sw_id': '5396522',
  'sw_name':

In [11]:
#look at the results for the table of interest
count = 0
swimmer_list = []
race_list = []
print("table header text", temp_table[2].find('caption').contents[0].split()[0])
for item in temp_table[2].find_all('tr'):
    temp_sw_dict = {}
    temp_rc_dict = {}
    if item.has_attr('class'):
        print("new swimmer!!")
        #print("row count", count, "length of info", len(item), "contents", item)
        #print("swimmer url", item.find('a').attrs['href'])
        #print("swimmer id", item.find('a').attrs['href'].split('/')[5])
        #print("swimmer name", item.find('a').contents[0])
        #print("swimmer YOB", item.find('th').contents[1][3:7])
        # need swim id for both the swimmer and the race dictionary
        sw_id = item.find('a').attrs['href'].split('/')[5]
        sw_yob = item.find('th').contents[1][3:7]
        temp_sw_dict['sw_url'] = item.find('a').attrs['href']
        temp_sw_dict['sw_id'] = sw_id
        temp_sw_dict['sw_name'] = item.find('a').contents[0]
        temp_sw_dict['sw_yob'] = sw_yob
        swimmer_list.append(temp_sw_dict)
    #print(count)
    #if item.children[0].has_attr('class'):
    #    print("race time and dist available")
    #    print(item.children[0])
    elif count != 0:
        #print("row count", count, len(item.contents))
        #for subItem in item:
        #    print(len(subItem), subItem.string)
        print("gender", item.contents[0].find('a').attrs['data-query-gender'])
        #print("race distance and stroke", item.contents[0])
        #print("race distance and stroke", item.contents[0].find('span').find('a').contents[0])
        #print("race round", item.contents[1].contents[0])
        #print("race time", item.contents[3].contents[0])
        #get the gender for both swim and race dict
        sw_gender = item.contents[0].find('a').attrs['data-query-gender']
        temp_rc_dict['sw_id'] = sw_id
        temp_rc_dict['sw_yob'] = sw_yob
        temp_rc_dict['sw_gender'] = sw_gender
        #temp_sw_dict['sw_gender'] = sw_gender
        temp_rc_dict['rc_dist_stroke'] = item.contents[0].find('span').find('a').contents[0]
        temp_rc_dict['rc_round'] = item.contents[1].contents[0]
        temp_rc_dict['rc_time'] = item.contents[3].contents[0]
        race_list.append(temp_rc_dict)
    else:
        print("row count", count)
        #print("table header text",item.caption)
    
    #append the temp dict to the lists
    #swimmer_list.append(temp_sw_dict)
    #race_list.append(temp_rc_dict)
    count += 1
    # just print out some to see the pattern
    #if count == 11:
    #    break

table header text Relays
row count 0
new swimmer!!


AttributeError: 'NoneType' object has no attribute 'attrs'

In [32]:
race_list[1]

{'sw_id': '5157731',
 'sw_yob': '1967',
 'sw_gender': 'X',
 'rc_dist_stroke': '200m Freestyle',
 'rc_round': 'Final',
 'rc_time': '2:26.41'}

In [ ]:
r2_url = meetList[0]['meet_url']
r2 = requests.get(r2_url)
temp_r2 = BeautifulSoup(r2.text, 'lxml')

My next step was to create a list of clubs that participated. I looked through the html on the page and determined the information I was interested in was held in 'option' tags.

In [ ]:
clubList = []
for item in temp_r2.find_all('option'):
    #print(item.contents[0])
    temp_dict = {}
    if "Events" in item.contents[0]:
        #print(item)
        break
    elif "Participants" not in item.contents[0]:
        temp_dict['club_res_url'] = item.attrs['data-href']
        temp_dict['club_number'] = item.attrs['value']
        temp_dict['club_name'] = item.contents[0]
        clubList.append(temp_dict)
        ## used for dev/debug
        #print(item.attrs['data-href'])
        #print(item.attrs['value'])
        #print(item.contents[0])
        
print("number of clubs in the meet", len(clubList))
print("information about each club") 
clubList[0] 

The next step is to get the club results and retrieve the information about each swimmer

In [ ]:
r3_url = clubList[0]['club_res_url']
r3 = requests.get(r3_url)
temp_r3 = BeautifulSoup(r3.text, 'lxml')

After examining the HTML for the per club results page, the results of interest seem to be in the first 4 tables:
* table 0 - men's results
* table 1 - women's results
* table 2 - relay results
* table 3 - meet resource info

In [ ]:
temp_table = temp_r3.find_all("table")
len(temp_table)

In [ ]:
#look at the results for the men's table
count = 0
swimmer_list = []
race_list = []
for item in temp_table[0].find_all('tr'):
    temp_sw_dict = {}
    temp_rc_dict = {}
    if item.has_attr('class'):
        print("new swimmer!!")
        #print("row count", count, "length of info", len(item), "contents", item)
        #print("swimmer url", item.find('a').attrs['href'])
        #print("swimmer id", item.find('a').attrs['href'].split('/')[5])
        #print("swimmer name", item.find('a').contents[0])
        #print("swimmer YOB", item.find('th').contents[1][3:7])
        # need swim id for both the swimmer and the race dictionary
        sw_id = item.find('a').attrs['href'].split('/')[5]
        sw_yob = item.find('th').contents[1][3:7]
        temp_sw_dict['sw_url'] = item.find('a').attrs['href']
        temp_sw_dict['sw_id'] = sw_id
        temp_sw_dict['sw_name'] = item.find('a').contents[0]
        temp_sw_dict['sw_yob'] = sw_yob
        swimmer_list.append(temp_sw_dict)
    #print(count)
    #if item.children[0].has_attr('class'):
    #    print("race time and dist available")
    #    print(item.children[0])
    elif count != 0:
        #print("row count", count, len(item.contents))
        #for subItem in item:
        #    print(len(subItem), subItem.string)
        #print("gender", item.contents[0].find('a').attrs['data-query-gender'])
        #print("race distance and stroke", item.contents[0])
        #print("race distance and stroke", item.contents[0].find('span').find('a').contents[0])
        #print("race round", item.contents[1].contents[0])
        #print("race time", item.contents[3].contents[0])
        #get the gender for both swim and race dict
        sw_gender = item.contents[0].find('a').attrs['data-query-gender']
        temp_rc_dict['sw_id'] = sw_id
        temp_rc_dict['sw_yob'] = sw_yob
        temp_rc_dict['sw_gender'] = sw_gender
        #temp_sw_dict['sw_gender'] = sw_gender
        temp_rc_dict['rc_dist_stroke'] = item.contents[0].find('span').find('a').contents[0]
        temp_rc_dict['rc_round'] = item.contents[1].contents[0]
        temp_rc_dict['rc_time'] = item.contents[3].contents[0]
        race_list.append(temp_rc_dict)
    else:
        print("row count", count)
    
    #append the temp dict to the lists
    #swimmer_list.append(temp_sw_dict)
    #race_list.append(temp_rc_dict)
    count += 1
    # just print out some to see the pattern
    #if count == 11:
    #    break

## Functions moved to utilities.py file

moving the functions here until I test the integration of utilities.py. Once I know it works, all the following code will be deleted.



In [ ]:
# function to get meet list
def getMeetList(season, month, base_url):
    #create param list for the URL
    call_params = {'season': season, 'province':'', 'month': month}
    
    #get the page after a random delay
    #I think the crawl delay is 10 seconds, so while it will take longer,
    #I will set a random delay of 10-14 seconds
    sleepTime = randint(10,14)
    logging.debug("sleep time is %i seconds", sleepTime)
    time.sleep(sleepTime)
    response=requests.get(base_url, params=call_params)
    logging.debug('%s', response.url)
    
    #parse the page and create the list
    #use Beautiful Soup to parse the returned page
    meetList_resp = BeautifulSoup(response.text, 'lxml')
    
    rtnList = []
    for item in meetList_resp.find_all('tr'):
        if item.contents[5].contents[0] == "Masters":
            ## need to handle the case where there was a meet, but there are no results
            
            temp_dict = {}
            try:
                temp_dict['meet_date'] = item.contents[0].contents[0].contents[0]
            except:
                temp_dict['meet_date'] = None
                
            try:    
                temp_dict['meet_url'] = item.contents[1].a.attrs['href']
            except:
                temp_dict['meet_url'] = None
                
            try:    
                temp_dict['meet_prov'] = item.contents[2].contents[0]
            except:
                temp_dict['meet_prov'] = None
                
            try:
                temp_dict['meet_host'] = item.contents[3].contents[0].contents[0]
            except:
                temp_dict['meet_host'] = None
                
            try:
                temp_dict['meet_course'] = item.contents[4].contents[0]
            except:
                temp_dict['meet_course'] = None
                
            try:
                temp_dict['meet_type'] = item.contents[5].contents[0]
            except:
                temp_dict['meet_type'] = None
                
            try:
                temp_dict['meet_status'] = item.contents[6].contents[0]
            except:
                temp_dict['meet_status'] = None
                
            rtnList.append(temp_dict)
    
    logging.debug("Number of meets: %i", len(rtnList))
    #return the list of meets
    return rtnList

In [ ]:
def getTeamList(meet_url):
    
    logging.debug(meet_url)
    sleepTime = randint(10,14)
    logging.debug("sleep time is %i seconds", sleepTime)
    time.sleep(sleepTime)
    response=requests.get(meet_url)
    logging.debug('%s', response.url)
        
    #parse the swim club list page
    #use Beautiful Soup to parse the returned page
    clubList_resp = BeautifulSoup(response.text, 'lxml')

    rtnList = []
    for item in clubList_resp.find_all('option'):
        temp_dict = {}
        if "Events" in item.contents[0]:
            logging.debug("Did I get here?")
            break
        elif "Participants" not in item.contents[0]:
            try:
                temp_dict['club_res_url'] = item.attrs['data-href']
            except:
                temp_dict['club_res_url'] = None
            try:
                temp_dict['club_number'] = item.attrs['value']
            except:
                temp_dict['club_number'] = None
            try:
                temp_dict['club_name'] = item.contents[0]
            except:
                temp_dict['club_name'] = None
            rtnList.append(temp_dict)

    return rtnList

In [ ]:
#todo: use this method with the previously define method rather than having the code repeated multiple times
def scrapePage(url):
    #get the page with the results for the club
    logging.debug(url)
    sleepTime = randint(10,14)
    logging.debug("sleep time is %i seconds", sleepTime)
    time.sleep(sleepTime)
    response=requests.get(url)
    logging.debug('%s', response.url)
    
    #use Beautiful Soup to parse the returned page
    resp = BeautifulSoup(response.text, 'lxml')
    
    #return the page
    return resp

In [ ]:
def parseSwimmerInfo(tableRow):
    
    #create return dictionary
    sw_dict = {}
    
    #parse the row
    try:
        sw_dict['sw_url'] = tableRow.find('a').attrs['href']
    except:
        sw_dict['sw_url'] = None 
    try:
        #sw_id = tableRow.find('a').attrs['href'].split('/')[5]
        sw_dict['sw_id'] = tableRow.find('a').attrs['href'].split('/')[5]
    except:
        sw_dict['sw_id'] = None
    try:
        sw_dict['sw_name'] = tableRow.find('a').contents[0]
    except:     
        sw_dict['sw_name'] = None
    try:
        sw_dict['sw_yob'] = tableRow.find('th').contents[1][3:7]
    except:    
        sw_dict['sw_yob'] = None
        
    #return the dictionary
    return sw_dict
        

In [ ]:
def parseRaceInfo(tableRow):
    
    #create return dictionary
    rc_dict = {}
    
    #rc_dict['sw_id'] = sw_id
    #rc_dict['sw_yob'] = sw_yob
    #try:
    #    rc_dict['sw_gender'] = tableRow.contents[0].find('a').attrs['data-query-gender']
    #except:
    #    rc_dict['sw_gender'] = None
    try:
        rc_dict['rc_dist_stroke'] = tableRow.contents[0].find('span').find('a').contents[0]
    except:
        rc_dict['rc_dist_stroke'] = None
    try:
        rc_dict['rc_round'] = tableRow.contents[1].contents[0]
    except:
        rc_dict['rc_round'] = None
    try:
        rc_dict['rc_time'] = tableRow.contents[3].contents[0]
    except:
        rc_dict['rc_time'] = None
    try:
        rc_dict['rc_course'] = tableRow.contents[4].find('abbr').contents[0]
    except:
        rc_dict['rc_course'] = None
    
    #print(rc_dict)
    return rc_dict

In [ ]:
##### Remember to change this back to use the URL as input
def getRaceResults(club_url):
#def getRaceResults(raceList_resp):
    
    ##### Remember to uncomment this to scrape the page
    #get the page with the results for the club
    raceList_resp = scrapePage(club_url)

    #create a list of tables on the page
    temp_table = raceList_resp.find_all('table')
    logging.debug("number of tables %i", len(temp_table))
    
    #create a gender table to use based on which table is being parsed
    gender_dict = {0:"male", 1:"female", 2:"relay"}
    
    #create lists to hold swimmer and race info
    swimmer_list = []
    race_list = []
    
    #if 3 or more tables, there are results on the page
    if len(temp_table) < 3:
        logging.info("no results on the page")
        #add code to put all nulls in the return value
    else:
        for tt in range(len(temp_table)-2):
            logging.debug("on table %i", tt)
            #set a flag for the first row, want to ignore it
            firstRow = True
            
            #get the gender based on which table is being parsed
            sw_gender = gender_dict[tt]
            
            #create variables to hold the last swimmer id and yob
            sw_id = None
            sw_yob = None
            
            #loop through each row in the table
            for item in temp_table[tt].find_all('tr'):
                temp_sw_dict = {}
                temp_rc_dict = {}
                if firstRow == True:
                    logging.debug("first row of table, ignore")
                    firstRow = False
                elif item.has_attr('class'):
                    #parse the swimmer information
                    temp_sw_dict = parseSwimmerInfo(item)
                    
                    #add the gender
                    try:
                        temp_sw_dict['sw_gender'] = sw_gender
                    except:
                        temp_sw_dict['sw_gender'] = None
                        
                    #make the swimmer id and yob available
                    sw_id = temp_sw_dict['sw_id']
                    sw_yob = temp_sw_dict['sw_yob']
                        
                    #append to the return list
                    swimmer_list.append(temp_sw_dict)
                    
                else:
                    
                    #parse the race information
                    temp_rc_dict = parseRaceInfo(item)
                    
                    #add the swimmer id, yob, gender
                    temp_rc_dict['sw_id'] = sw_id
                    temp_rc_dict['sw_yob'] = sw_yob
                    temp_rc_dict['sw_gender'] = sw_gender
                    
                    #append to the return list
                    race_list.append(temp_rc_dict)
                    
                #else:
                #    logging.debug("expect no information in this row")
                #count += 1
    return swimmer_list, race_list

In [9]:
clubURLS[0]

'https://www.swimming.ca/en/meet/604494/?faction=73992'

In [10]:
#test my utils file
tmp_sw, tmp_rc = util.getRaceResults(clubURLS[0])

DEBUG:root:https://www.swimming.ca/en/meet/604494/?faction=73992
DEBUG:root:sleep time is 10 seconds
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.swimming.ca:443
DEBUG:urllib3.connectionpool:https://www.swimming.ca:443 "GET /en/meet/604494/?faction=73992 HTTP/1.1" 200 None
DEBUG:root:https://www.swimming.ca/en/meet/604494/?faction=73992
DEBUG:root:number of tables 5
DEBUG:root:on table 0
DEBUG:root:first row of table, ignore
DEBUG:root:on table 1
DEBUG:root:first row of table, ignore
DEBUG:root:on table 2
DEBUG:root:first row of table, ignore


In [12]:
tmp_sw[0]

{'sw_url': 'https://www.swimming.ca/en/swimmer/4209372/',
 'sw_id': '4209372',
 'sw_name': 'Aviles, Carlos',
 'sw_yob': '1975',
 'sw_gender': 'male'}

In [13]:
tmp_rc[0]

{'rc_dist_stroke': '50m Freestyle',
 'rc_round': 'Final',
 'rc_time': '29.04',
 'rc_course': '25m',
 'sw_id': '4209372',
 'sw_yob': '1975',
 'sw_gender': 'male'}

In [ ]:
#for testing, I am going to get 2 pages - one with relays and one without to make sure I can handle either
url_relays = 'https://www.swimming.ca/en/meet/598224/?faction=73928'
url_no_relays = 'https://www.swimming.ca/en/meet/597428/?faction=74044'
relay_pg = scrapePage(url_relays)
no_relay_pg = scrapePage(url_no_relays)
